In [1]:
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, Dataset # PyTorch의 Dataset 클래스를 상속받기 위한 모듈
import matplotlib.patches as patches
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import os              # 파일 및 디렉토리 경로를 다루기 위한 표준 라이브러리
import json            # JSON 파일을 읽고 쓰기 위한 표준 라이브러리
import torch           # 딥러닝 라이브러리 PyTorch (텐서 연산 등)
from PIL import Image  # 이미지를 다루기 위한 Pillow 라이브러리
from torchvision import transforms
from pathlib import Path
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.optim import Adam
import torch.nn as nn
import matplotlib
from pathlib import Path  # payhon path
import albumentations as A
import cv2  # OpenCV - 고급 이미지/비디오 처리
import pandas as pd
from tqdm import tqdm
!pip install torchmetrics
from torchmetrics.detection.mean_ap import MeanAveragePrecision
# Garbage Collector 모듈
import gc

In [2]:
# 메모리 정리 루틴
gc.collect()
torch.cuda.empty_cache()

# --- 디바이스 설정 ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42

# --- 학습 하이퍼파라미터 ---
BATCH_SIZE = 16
NUM_EPOCHS = 20
LEARNING_RATE = 1e-4
MOMENTUM = 0.9
WEIGHT_DECAY = 0.0005

# --- 모델 설정 ---
NUM_CLASSES = 73
MODEL_NAME = "fasterrcnn_resnet50_fpn"
USE_PRETRAINED = True

# --- 학습 고도화 설정 ---
USE_SCHEDULER = True  # Learning rate scheduler 사용 여부
EARLY_STOPPING = True  # Early stopping 적용 여부
AUGMENTATION = True  # 데이터 증강 사용 여부

# --- 실험 로깅용 설정 ---
USE_WANDB = True
WANDB_PROJECT = "AI03-Project-1"
RUN_NAME = f"{MODEL_NAME}_bs{BATCH_SIZE}_lr{LEARNING_RATE}"


# --- 실험 결과 저장 경로 ---
EXPERIMENT_DIR = "../experiments"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os

project_path = '/content/drive/MyDrive/data/raw'
os.chdir(project_path)

print("현재 디렉토리:", os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
현재 디렉토리: /content/drive/MyDrive/data/raw


In [4]:
import os
from collections import defaultdict

def analyze_directory(path, name=""):
    print(f"\n [{name}] - 경로: {path}")

    if not os.path.exists(path):
        print(" 존재하지 않음")
        return

    all_files = []
    file_ext_counter = defaultdict(int)

    for root, _, files in os.walk(path):
        for file in files:
            all_files.append(os.path.join(root, file))
            ext = os.path.splitext(file)[1].lower()
            file_ext_counter[ext] += 1

    print(f"총 파일 수: {len(all_files)}")
    print("파일 확장자 분포:")
    for ext, count in file_ext_counter.items():
        print(f"  - {ext or '[no extension]'}: {count}개")

    print("샘플:")
    for f in all_files[:5]:
        print(f"  └ {f}")

# 디렉토리 경로 설정
train_images_path = './train_images'
test_images_path = './test_images'
train_annots_path = './train_annotations'

# 분석 실행
analyze_directory(train_images_path, name="Train Images")
analyze_directory(test_images_path, name="Test Images")
analyze_directory(train_annots_path, name="Train Annotations (including subfolders)")



 [Train Images] - 경로: ./train_images
총 파일 수: 1489
파일 확장자 분포:
  - .png: 1489개
샘플:
  └ ./train_images/K-003351-013900-016232_0_2_0_2_90_000_200.png
  └ ./train_images/K-003351-013900-022074_0_2_0_2_70_000_200.png
  └ ./train_images/K-003351-013900-022074_0_2_0_2_75_000_200.png
  └ ./train_images/K-003351-016232-019232_0_2_0_2_75_000_200.png
  └ ./train_images/K-003351-013900-035206_0_2_0_2_75_000_200.png

 [Test Images] - 경로: ./test_images
총 파일 수: 843
파일 확장자 분포:
  - .png: 843개
샘플:
  └ ./test_images/1013.png
  └ ./test_images/1012.png
  └ ./test_images/100.png
  └ ./test_images/1007.png
  └ ./test_images/10.png

 [Train Annotations (including subfolders)] - 경로: ./train_annotations
총 파일 수: 4531
파일 확장자 분포:
  - [no extension]: 4개
  - .json: 4527개
샘플:
  └ ./train_annotations/.DS_Store
  └ ./train_annotations/K-003483-027733-028763-036637_json/K-027733/K-003483-027733-028763-036637_0_2_0_2_75_000_200.json
  └ ./train_annotations/K-003483-027733-028763-036637_json/K-027733/K-003483-027733-0287

In [5]:
import os
import pandas as pd
import json

# 경로 설정
train_image_dir = './train_images'
test_image_dir = './test_images'
annotation_root = '/content/drive/MyDrive/data/raw/train_annotations'

# 1. Train Images → DataFrame
train_image_list = [
    {
        'image_id': f,
        'image_path': os.path.join(train_image_dir, f)
    }
    for f in os.listdir(train_image_dir)
    if f.endswith('.png')
]
df_train_images = pd.DataFrame(train_image_list)

# 2. Test Images → DataFrame
test_image_list = [
    {
        'image_id': f,
        'image_path': os.path.join(test_image_dir, f)
    }
    for f in os.listdir(test_image_dir)
    if f.endswith('.png')
]
df_test_images = pd.DataFrame(test_image_list)

In [6]:
import os
import json
import pandas as pd

annotation_root = '/content/drive/MyDrive/data/raw/train_annotations'
train_image_dir = '/content/drive/MyDrive/data/raw/train_images'

annotation_data = []

for folder1 in os.listdir(annotation_root):
    folder1_path = os.path.join(annotation_root, folder1)
    if not os.path.isdir(folder1_path):
        continue

    for folder2 in os.listdir(folder1_path):
        folder2_path = os.path.join(folder1_path, folder2)
        if not os.path.isdir(folder2_path):
            continue

        for filename in os.listdir(folder2_path):
            if filename.endswith('.json'):
                json_path = os.path.join(folder2_path, filename)

                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        ann = json.load(f)

                    image_info = ann['images'][0]
                    image_file = image_info['file_name']
                    image_path = os.path.join(train_image_dir, image_file)
                    image_id = image_info['id']
                    width = image_info['width']
                    height = image_info['height']

                    category_map = {cat['id']: cat['name'] for cat in ann['categories']}

                    for obj in ann.get('annotations', []):
                        x, y, w, h = obj['bbox']
                        annotation_data.append({
                            'image_id': image_id,
                            'image_file': image_file,
                            'image_path': image_path,
                            'class_name': category_map.get(obj['category_id'], 'Unknown'),
                            'x_min': x,
                            'y_min': y,
                            'x_max': x + w,
                            'y_max': y + h,
                            'width': width,
                            'height': height
                        })

                except Exception as e:
                    print(f"{json_path} 읽는 중 에러 발생: {e}")

df_annotations = pd.DataFrame(annotation_data)


In [7]:
df_annotations.head()

,image_id,image_file,image_path,class_name,x_min,y_min,x_max,y_max,width,height
0,20,K-003483-027733-028763-036637_0_2_0_2_75_000_2...,/content/drive/MyDrive/data/raw/train_images/K...,트윈스타정 40/5mg,88,728,408,1010,976,1280
1,21,K-003483-027733-028763-036637_0_2_0_2_70_000_2...,/content/drive/MyDrive/data/raw/train_images/K...,트윈스타정 40/5mg,562,262,865,529,976,1280
2,19,K-003483-027733-028763-036637_0_2_0_2_90_000_2...,/content/drive/MyDrive/data/raw/train_images/K...,트윈스타정 40/5mg,125,767,441,1044,976,1280
3,21,K-003483-027733-028763-036637_0_2_0_2_70_000_2...,/content/drive/MyDrive/data/raw/train_images/K...,트라젠타정(리나글립틴),675,896,907,1111,976,1280
4,20,K-003483-027733-028763-036637_0_2_0_2_75_000_2...,/content/drive/MyDrive/data/raw/train_images/K...,트라젠타정(리나글립틴),85,166,295,365,976,1280


In [8]:
df_annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4527 entries, 0 to 4526
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_id    4527 non-null   int64 
 1   image_file  4527 non-null   object
 2   image_path  4527 non-null   object
 3   class_name  4527 non-null   object
 4   x_min       4527 non-null   int64 
 5   y_min       4527 non-null   int64 
 6   x_max       4527 non-null   int64 
 7   y_max       4527 non-null   int64 
 8   width       4527 non-null   int64 
 9   height      4527 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 353.8+ KB


In [9]:
df_annotations.describe()

,image_id,x_min,y_min,x_max,y_max,width,height
count,4527.000000,4527.000000,4527.000000,4527.000000,4527.000000,4527.0,4527.0
mean,751.360945,356.738679,484.161476,616.062956,772.943450,976.0,1280.0
std,432.269700,256.591048,328.961787,252.410652,325.507638,0.0,0.0
min,1.000000,0.000000,0.000000,240.000000,283.000000,976.0,1280.0
25%,376.000000,121.000000,191.000000,387.000000,464.000000,976.0,1280.0
50%,757.000000,379.000000,543.000000,567.000000,914.000000,976.0,1280.0
75%,1124.500000,598.000000,796.000000,848.000000,1053.000000,976.0,1280.0
max,1500.000000,6567.000000,8889.000000,6878.000000,9106.000000,976.0,1280.0


In [10]:
df_annotations = df_annotations[
    (df_annotations['x_min'] >= 0) &
    (df_annotations['y_min'] >= 0) &
    (df_annotations['x_max'] <= 976) &
    (df_annotations['y_max'] <= 1280) &
    (df_annotations['x_max'] > df_annotations['x_min']) &
    (df_annotations['y_max'] > df_annotations['y_min'])
]

In [11]:
df_annotations.describe()

,image_id,x_min,y_min,x_max,y_max,width,height
count,4525.000000,4525.000000,4525.000000,4525.000000,4525.000000,4525.0,4525.0
mean,751.571713,355.300773,482.272928,614.622983,771.064972,976.0,1280.0
std,432.248273,239.419191,304.369768,234.638992,301.070016,0.0,0.0
min,1.000000,0.000000,0.000000,240.000000,283.000000,976.0,1280.0
25%,377.000000,121.000000,191.000000,387.000000,464.000000,976.0,1280.0
50%,757.000000,379.000000,539.000000,567.000000,910.000000,976.0,1280.0
75%,1125.000000,598.000000,796.000000,848.000000,1053.000000,976.0,1280.0
max,1500.000000,709.000000,932.000000,975.000000,1271.000000,976.0,1280.0


In [12]:
# 1. bbox 크기 계산
df_annotations["bbox_width"] = df_annotations["x_max"] - df_annotations["x_min"]
df_annotations["bbox_height"] = df_annotations["y_max"] - df_annotations["y_min"]
df_annotations["bbox_area"] = df_annotations["bbox_width"] * df_annotations["bbox_height"]

# 2. 너무 작은 bbox 제거 (너비 또는 높이 30 미만)
small_bbox_condition = (df_annotations["bbox_width"] < 30) | (df_annotations["bbox_height"] < 30)

# 3. 너무 큰 bbox 제거 (전체 이미지의 90% 이상 차지하는 경우)
too_large_area_condition = df_annotations["bbox_area"] > (df_annotations["width"] * df_annotations["height"] * 0.9)

# 4. 이상치 필터링
df_bbox_filtered = df_annotations[~(small_bbox_condition | too_large_area_condition)].reset_index(drop=True)


/tmp/ipython-input-12-1710214241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_annotations["bbox_width"] = df_annotations["x_max"] - df_annotations["x_min"]
/tmp/ipython-input-12-1710214241.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_annotations["bbox_height"] = df_annotations["y_max"] - df_annotations["y_min"]
/tmp/ipython-input-12-1710214241.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [13]:
print(f"Before: {len(df_annotations)} → After: {len(df_bbox_filtered)}")
df_bbox_filtered.describe()

Before: 4525 → After: 4525


,image_id,x_min,y_min,x_max,y_max,width,height,bbox_width,bbox_height,bbox_area
count,4525.000000,4525.000000,4525.000000,4525.000000,4525.000000,4525.0,4525.0,4525.000000,4525.000000,4525.000000
mean,751.571713,355.300773,482.272928,614.622983,771.064972,976.0,1280.0,259.322210,288.792044,78894.418564
std,432.248273,239.419191,304.369768,234.638992,301.070016,0.0,0.0,70.033259,116.984440,47206.376994
min,1.000000,0.000000,0.000000,240.000000,283.000000,976.0,1280.0,125.000000,123.000000,18492.000000
25%,377.000000,121.000000,191.000000,387.000000,464.000000,976.0,1280.0,209.000000,199.000000,43382.000000
50%,757.000000,379.000000,539.000000,567.000000,910.000000,976.0,1280.0,242.000000,231.000000,57452.000000
75%,1125.000000,598.000000,796.000000,848.000000,1053.000000,976.0,1280.0,295.000000,403.000000,106330.000000
max,1500.000000,709.000000,932.000000,975.000000,1271.000000,976.0,1280.0,529.000000,669.000000,272435.000000


In [14]:
# 중복 여부 확인
duplicate_mask = df_annotations.duplicated(subset=['image_id', 'x_min', 'y_min', 'x_max', 'y_max', 'class_name'], keep=False)

# 중복된 bbox만 추출
df_duplicates = df_annotations[duplicate_mask].sort_values(by='image_id')

print(f"중복된 bbox 수: {len(df_duplicates)}")
df_duplicates.head()

중복된 bbox 수: 2


,image_id,image_file,image_path,class_name,x_min,y_min,x_max,y_max,width,height,bbox_width,bbox_height,bbox_area
2243,737,K-001900-016551-031705-044199_0_2_0_2_90_000_2...,/content/drive/MyDrive/data/raw/train_images/K...,낙소졸정 500/20mg,171,679,420,1131,976,1280,249,452,112548
2244,737,K-001900-016551-031705-044199_0_2_0_2_90_000_2...,/content/drive/MyDrive/data/raw/train_images/K...,낙소졸정 500/20mg,171,679,420,1131,976,1280,249,452,112548


In [15]:
df_train_images.head()

,image_id,image_path
0,K-003351-013900-016232_0_2_0_2_90_000_200.png,./train_images/K-003351-013900-016232_0_2_0_2_...
1,K-003351-013900-022074_0_2_0_2_70_000_200.png,./train_images/K-003351-013900-022074_0_2_0_2_...
2,K-003351-013900-022074_0_2_0_2_75_000_200.png,./train_images/K-003351-013900-022074_0_2_0_2_...
3,K-003351-016232-019232_0_2_0_2_75_000_200.png,./train_images/K-003351-016232-019232_0_2_0_2_...
4,K-003351-013900-035206_0_2_0_2_75_000_200.png,./train_images/K-003351-013900-035206_0_2_0_2_...


In [16]:
df_train_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_id    1489 non-null   object
 1   image_path  1489 non-null   object
dtypes: object(2)
memory usage: 23.4+ KB


In [17]:
df_test_images.head()

,image_id,image_path
0,1013.png,./test_images/1013.png
1,1012.png,./test_images/1012.png
2,100.png,./test_images/100.png
3,1007.png,./test_images/1007.png
4,10.png,./test_images/10.png


In [18]:
df_test_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 843 entries, 0 to 842
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_id    843 non-null    object
 1   image_path  843 non-null    object
dtypes: object(2)
memory usage: 13.3+ KB


In [19]:
import os

train_img_dir = "./train_images"
test_img_dir = "./test_images"

In [20]:
# 기존에 만든 df_annotations에서 image_file 컬럼 사용
ann_image_files = set(df_annotations["image_file"].tolist())

print(f"annotations에 쓰인 이미지 수: {len(ann_image_files)}")

annotations에 쓰인 이미지 수: 1489


In [21]:
train_image_files = set(os.listdir(train_img_dir))
test_image_files = set(os.listdir(test_img_dir))

print(f"train_images 폴더 이미지 수: {len(train_image_files)}")
print(f"test_images 폴더 이미지 수: {len(test_image_files)}")

train_images 폴더 이미지 수: 1489
test_images 폴더 이미지 수: 843


In [22]:
# annotations에 쓰인 이미지가 train_images/test_images에도 있는지 확인
ann_train_overlap = ann_image_files & train_image_files
ann_test_overlap = ann_image_files & test_image_files

print(f"annotations ∩ train_images 중복 수: {len(ann_train_overlap)}")
print(f"annotations ∩ test_images 중복 수: {len(ann_test_overlap)}")

annotations ∩ train_images 중복 수: 1489
annotations ∩ test_images 중복 수: 0


In [23]:
print("train 중복 샘플:", list(ann_train_overlap)[:5])
print("test 중복 샘플:", list(ann_test_overlap)[:5])

train 중복 샘플: ['K-002483-003743-004378-012778_0_2_0_2_70_000_200.png', 'K-003483-027733-029667-030308_0_2_0_2_70_000_200.png', 'K-002483-005094-013395-019552_0_2_0_2_75_000_200.png', 'K-003483-016262-025367-027777_0_2_0_2_70_000_200.png', 'K-003351-018357-036637_0_2_0_2_75_000_200.png']
test 중복 샘플: []


In [24]:
import os
from PIL import Image

def find_broken_images(image_dir):
    broken_images = []
    all_images = os.listdir(image_dir)

    for img_name in all_images:
        img_path = os.path.join(image_dir, img_name)

        try:
            with Image.open(img_path) as img:
                img.verify()  # 이미지가 제대로 열리는지 확인만 (메모리에 로딩 X)
        except Exception as e:
            print(f"깨진 이미지 발견: {img_path} | 에러: {e}")
            broken_images.append(img_path)

    return broken_images

train_image_dir = "./train_images"
test_image_dir = "./test_images"

broken_train = find_broken_images(train_image_dir)
broken_test = find_broken_images(test_image_dir)

print(f"\nTrain에서 깨진 이미지 수: {len(broken_train)}")
print(f" Test에서 깨진 이미지 수: {len(broken_test)}")


Train에서 깨진 이미지 수: 0
 Test에서 깨진 이미지 수: 0


In [25]:
# 이미지당 객체 개수 세기
bbox_per_image = df_annotations.groupby('image_file').size().reset_index(name='num_bboxes')

# 4개 초과하는 이미지 확인
over_limit = bbox_per_image[bbox_per_image['num_bboxes'] > 4]

# 결과 출력
print(f"전체 이미지 수: {bbox_per_image.shape[0]}")
print(f"4개 초과 bbox 가진 이미지 수: {over_limit.shape[0]}")
display(over_limit.head())

전체 이미지 수: 1489
4개 초과 bbox 가진 이미지 수: 1


,image_file,num_bboxes
215,K-001900-016551-031705-044199_0_2_0_2_90_000_2...,5


In [26]:
def parse_raw_annotations(ann_dir: Path) -> pd.DataFrame:
    """
    복잡한 3중 폴더 구조의 원본 어노테이션을 파싱하여
    하나의 Pandas DataFrame으로 반환하는 함수.
    """
    all_annotations = []

    # Level 1: 이미지별 폴더 순회
    image_level_dirs = os.listdir(ann_dir)
    for image_dir_name in tqdm(image_level_dirs, desc="[L1] Images"):
        image_dir_path = ann_dir / image_dir_name
        if not image_dir_path.is_dir():
            continue

        # Level 2: 알약 종류 폴더 순회
        pill_level_dirs = os.listdir(image_dir_path)
        for pill_dir_name in pill_level_dirs:
            pill_dir_path = image_dir_path / pill_dir_name
            if not pill_dir_path.is_dir():
                continue

            # Level 3: 실제 .json 파일 파싱
            json_files = [f for f in os.listdir(pill_dir_path) if f.endswith(".json")]
            if not json_files:
                continue

            # 첫 번째 json 파일만 사용
            json_file_path = pill_dir_path / json_files[0]

            try:
                with open(json_file_path, "r", encoding="utf-8") as f:
                    ann_data = json.load(f)

                    image_info = ann_data.get("images", [{}])[0]
                    annotation_info = ann_data.get("annotations", [{}])[0]
                    category_info = ann_data.get("categories", [{}])[0]

                    all_annotations.append(
                        {
                            "image_id": image_info.get("id"),
                            "file_name": image_info.get("file_name"),
                            "width": image_info.get("width"),
                            "height": image_info.get("height"),
                            "category_id": category_info.get("id"),
                            "class_name": category_info.get("name"),
                            "bbox": annotation_info.get("bbox"),
                        }
                    )
            except Exception as e:
                print(f"\n파일 처리 에러: {json_file_path}, 에러: {e}")

    return pd.DataFrame(all_annotations)

In [27]:
# 재현성을 위해 모든 난수 생성기의 시드를 고정하는 함수.
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Seed set to {seed}")

In [28]:
# 디바이스 확인
print(f"Using device: {DEVICE}")

Using device: cuda


In [29]:
# 1. 핵심 함수를 호출해서 DataFrame 생성
from pathlib import Path

BASE_DIR = Path("/content/drive/MyDrive/data/")
RAW_DATA_DIR = BASE_DIR / "raw"
PROCESSED_DATA_DIR = BASE_DIR / "data/processed"

TRAIN_IMAGE_DIR = RAW_DATA_DIR / "train_images"
TRAIN_ANNO_DIR = RAW_DATA_DIR / "train_annotations"
TEST_IMAGE_DIR = RAW_DATA_DIR / "test_images"
SAVE_PATH = PROCESSED_DATA_DIR / "train_df.csv"
train_df = parse_raw_annotations(TRAIN_ANNO_DIR)


# --- (1). bbox 컬럼을 4개로 분리 ---
# bbox 컬럼 분리
bbox_df = pd.DataFrame(
    train_df["bbox"].tolist(), columns=["bbox_x", "bbox_y", "bbox_w", "bbox_h"]
)
train_df = pd.concat([train_df.drop("bbox", axis=1), bbox_df], axis=1)

# ✨ --- [핵심 수정] 잘못된 바운딩 박스 데이터 제거 ---
# xmax (bbox_x + bbox_w)가 이미지 너비(width)를 초과하는 경우
invalid_x = train_df["bbox_x"] + train_df["bbox_w"] > train_df["width"]
# ymax (bbox_y + bbox_h)가 이미지 높이(height)를 초과하는 경우
invalid_y = train_df["bbox_y"] + train_df["bbox_h"] > train_df["height"]

# 잘못된 데이터를 필터링
invalid_rows = train_df[invalid_x | invalid_y]
if not invalid_rows.empty:
    print(f"--- {len(invalid_rows)}개의 잘못된 바운딩 박스 데이터를 찾았습니다. ---")
    print(
        invalid_rows[
            ["file_name", "width", "height", "bbox_x", "bbox_y", "bbox_w", "bbox_h"]
        ]
    )

    # 유효한 데이터만 남김
    train_df = train_df[~(invalid_x | invalid_y)]
    print(f"\n잘못된 데이터를 제거하고, {len(train_df)}개의 데이터만 사용합니다.")

# --- (2). category_id를 새로운 label_idx로 매핑 ---
# 고유한 category_id 목록을 뽑아 정렬
unique_category_ids = sorted(train_df["category_id"].unique())
NUM_CLASSES = len(unique_category_ids)
# category_id를 0, 1, 2... 인덱스로 변환하는 딕셔너리 생성
id_to_idx = {
    int(original_id): idx
    for idx, original_id in enumerate(
        unique_category_ids, start=1
    )  # <--- start=1 추가!
}
# 이 매핑 정보를 사용해서 'label_idx'라는 새 컬럼을 추가
train_df["label_idx"] = train_df["category_id"].map(id_to_idx)
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)


# 나중에 추론 결과에서 원래 클래스 이름을 찾을 수 있도록 매핑 정보도 저장
label_map = {
    "id_to_idx": id_to_idx,
    "idx_to_id": {idx: int(original_id) for original_id, idx in id_to_idx.items()},
    "id_to_name": dict(zip(train_df["category_id"], train_df["class_name"])),
}
with open(PROCESSED_DATA_DIR / "label_map.json", "w", encoding="utf-8") as f:
    json.dump(label_map, f, ensure_ascii=False, indent=4)

print(f"\n총 {len(unique_category_ids)}개의 고유 클래스를 발견했습니다.")
print("라벨 매핑 정보를 'data/processed/label_map.json'에 저장했습니다.")


# 3. 최종 DataFrame을 CSV 파일로 저장
train_df.to_csv(SAVE_PATH, index=False)

print(f"\n--- 데이터 전처리 및 저장 완료! ---")
print(train_df.head())

[L1] Images: 100%|██████████| 499/499 [00:04<00:00, 113.40it/s]


총 73개의 고유 클래스를 발견했습니다.
라벨 매핑 정보를 'data/processed/label_map.json'에 저장했습니다.

--- 데이터 전처리 및 저장 완료! ---
   image_id                                          file_name  width  height  \
0        20  K-003483-027733-028763-036637_0_2_0_2_75_000_2...    976    1280   
1        21  K-003483-027733-028763-036637_0_2_0_2_70_000_2...    976    1280   
2        21  K-003483-027733-028763-036637_0_2_0_2_70_000_2...    976    1280   
3        19  K-003483-027733-028763-036637_0_2_0_2_90_000_2...    976    1280   
4        23  K-003483-016262-025367-027653_0_2_0_2_75_000_2...    976    1280   

   category_id          class_name  bbox_x  bbox_y  bbox_w  bbox_h  label_idx  
0        27732        트윈스타정 40/5mg      88     728     320     282         50  
1        28762        트라젠타정(리나글립틴)     675     896     232     215         54  
2        36636        로수젯정10/5밀리그램     100     884     270     179         70  
3         3482  기넥신에프정(은행엽엑스)(수출용)     612     787     272     213          4  
4        253

In [30]:
# (1) 데이터 증강 (Augmentation) : Albumentations 라이브러리 사용
train_transforms = A.Compose(
    [
        A.Resize(512, 512),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        # PyTorch 텐서로 변환
        A.pytorch.ToTensorV2(),
    ],
    bbox_params=A.BboxParams(format="albumentations", label_fields=["labels"]),
)  # bbox 형식은 pascal_voc: [xmin, ymin, xmax, ymax]

val_transforms = A.Compose(
    [
        A.Resize(512, 512),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.pytorch.ToTensorV2(),
    ],
    bbox_params=A.BboxParams(format="albumentations", label_fields=["labels"]),
)

test_transforms = A.Compose(
    [
        A.Resize(512, 512),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.pytorch.ToTensorV2(),
    ]
)


# DataLoader를 위한 collate_fn. 이미지와 타겟을 리스트로 묶어줌
def collate_fn(batch):
    return tuple(zip(*batch))

In [31]:
class PillDataset(Dataset):
    # --- mode 파라미터 추가 및 df를 직접 받도록 수정 ---
    def __init__(self, df, image_dir, mode="train", transforms=None):
        self.df = df
        self.image_dir = Path(image_dir)
        self.mode = mode
        self.transforms = transforms

        # --- image_ids를 미리 뽑아 중복을 제거 ---
        # df['file_name']을 사용하면 이미지 파일 이름으로 고유한 이미지를 식별 가능.
        self.image_ids = self.df["file_name"].unique()

    def __len__(self):
        # --- 고유한 이미지의 개수를 반환 ---
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_path = self.image_dir / image_id

        image = cv2.imread(str(image_path))
        if image is None:
            raise FileNotFoundError(
                f"Error: Could not load image at path: {image_path}"
            )

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape

        if self.mode in ["train", "val"]:
            records = self.df[self.df["file_name"] == image_id]
            boxes = records[["bbox_x", "bbox_y", "bbox_w", "bbox_h"]].values

            boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
            boxes[:, 3] = boxes[:, 1] + boxes[:, 3]

            labels = records["label_idx"].values

            # print(f"\n[DEBUG 1] Image: {image_id}, Original Pixel Coords:\n{boxes}")

            # 바운딩 박스 좌표 정규화
            boxes = boxes.astype(np.float32)
            boxes[:, [0, 2]] /= w
            boxes[:, [1, 3]] /= h

            # print(f"[DEBUG 2] Normalized Coords for Albumentations:\n{boxes}")

            if self.transforms:
                try:
                    transformed = self.transforms(
                        image=image, bboxes=boxes, labels=labels
                    )
                    image = transformed["image"]
                    boxes = transformed["bboxes"]
                    labels = transformed["labels"]
                except Exception as e:
                    print(f"!!!!!!!!!!!!!! Albumentations에서 에러 발생 !!!!!!!!!!!!!!")
                    print(f"Image: {image_id}")
                    print(f"Boxes sent to transform: {boxes}")
                    # raise e  # 에러를 다시 발생시켜서 멈추게 함

            # ... 이하 코드는 이전과 동일 ...
            _, new_h, new_w = image.shape
            boxes = torch.as_tensor(boxes, dtype=torch.float32)

            if len(boxes) > 0:
                boxes[:, [0, 2]] *= new_w
                boxes[:, [1, 3]] *= new_h

            target = {
                "boxes": boxes,
                "labels": torch.as_tensor(labels, dtype=torch.int64),
            }

            if len(target["boxes"]) == 0:
                target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
                target["labels"] = torch.zeros((0,), dtype=torch.int64)

            return image, target

            # 테스트 모드일 경우, 이미지와 파일 이름만 반환
        elif self.mode == "test":
            # 테스트 시에는 보통 기본적인 리사이즈, 정규화만 적용
            if self.transforms:
                transformed = self.transforms(image=image)
                image = transformed["image"]

            # 나중에 예측 결과를 이미지와 매칭시키기 위해 파일 이름을 반환
            return image, image_id


# 참고: Subset을 사용할 때 transform을 다르게 적용하려면 약간의 트릭이 필요.
# 먼저 transform이 없는 전체 데이터셋을 만듦.
# 각 Subset에 맞는 transform을 적용하는 Wrapper 클래스 생성
# class TransformSubset(Dataset):
#     def __init__(self, subset, transforms):
#         self.subset = subset
#         self.transforms = transforms

#     def __getitem__(self, idx):
#         image, target = self.subset[idx]

#         # NumPy 배열로 변환 (Albumentations 입력 형식)
#         boxes = target["boxes"].numpy()
#         labels = target["labels"].numpy()

#         if self.transforms:
#             transformed = self.transforms(image=image, bboxes=boxes, labels=labels)
#             image = transformed["image"]
#             target["boxes"] = torch.as_tensor(
#                 transformed["bboxes"], dtype=torch.float32
#             )
#             # 증강 후 bbox가 사라졌을 경우 처리
#             if len(target["boxes"]) == 0:
#                 target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)

#         return image, target

#     def __len__(self):
#         return len(self.subset)

# 모델 학습
학습에 사용된 모델은 torchvision.models.detection 라이브러리의 fasterrcnn_resnet50_fpn 모델입니다. 해당 모델은 resnet - 50을 backbone network로 하여 fpn기법을 통해 5개의 feature map을 산출하고, 이를 detection task에 활용하는 모델입니다.

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
# pre-trained 모델 로드
# Faster R-CNN

# Anchor 크기 설정 (작은 객체 위주)
anchor_sizes = ((8,), (16,), (24,), (32,), (40,))

# 비율 설정 (표준적인 3가지)
aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)

# Anchor Generator
anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

# 모델 정의
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1,
    rpn_anchor_generator=anchor_generator,
    min_size=350,  # 작은 객체를 위한 이미지 크기
    box_batch_size_per_image=100  # proposal 수 조절
)


model.rpn.pre_nms_top_n['training'] = 2000
model.rpn.post_nms_top_n['training'] = 1000

# 분류기의 입력 피처 수를 가져옴
in_features = model.roi_heads.box_predictor.cls_score.in_features

# pre-trained head를 새로운 head로 교체
# num_classes에 배경(background) 클래스 1개를 더해줘야 함
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES + 1)

In [33]:
import random


# 재현성을 위해 모든 난수 생성기의 시드를 고정하는 함수.
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Seed set to {seed}")

In [34]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedGroupKFold
from torch.utils.data import Subset

seed_everything(SEED)

# 1. 데이터 준비
df = pd.read_csv(SAVE_PATH)

# StratifiedGroupKFold를 위한 데이터 준비
groups = df["file_name"]  # 그룹 기준: 이미지 파일 이름
labels = df["category_id"]  # 층화 기준: 원본 클래스 ID

# K-Fold 설정 (5-fold, 즉 80% train / 20% val)
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=SEED)

# 첫 번째 fold의 train/val 인덱스를 가져옴
train_idxs, val_idxs = next(cv.split(df, labels, groups))
# 1. 인덱스를 사용해서 데이터프레임을 먼저 분할!
train_df_split = df.iloc[train_idxs].reset_index(drop=True)
val_df_split = df.iloc[val_idxs].reset_index(drop=True)

# 2. 분할된 데이터프레임으로 각각 Dataset 생성 (Subset, TransformSubset 불필요!)
train_dataset = PillDataset(
    df=train_df_split,
    image_dir=TRAIN_IMAGE_DIR,
    mode="train",
    transforms=train_transforms,
)

val_dataset = PillDataset(
    df=val_df_split,
    image_dir=TRAIN_IMAGE_DIR,
    mode="val",
    transforms=val_transforms,  # val_transforms 사용
)


test_df = pd.DataFrame({"file_name": os.listdir(TEST_IMAGE_DIR)})

test_dataset = PillDataset(
    df=test_df,
    image_dir=TEST_IMAGE_DIR,
    mode="test",
    transforms=test_transforms,
)


# --- Data Loader ---
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
)

Seed set to 42


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:1023: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
img, image_id = next(iter(val_loader))
image_id[6]

{'boxes': tensor([[353.5738, 109.2000, 446.9508, 178.8000]]),
 'labels': tensor([36])}

In [36]:
model = model.to(DEVICE)

params = [p for p in model.parameters() if p.requires_grad]

# optimizer = torch.optim.SGD(
#     params,
#     lr=LEARNING_RATE,
#     momentum=MOMENTUM,
#     weight_decay=WEIGHT_DECAY,
# )
optimizer = torch.optim.AdamW(params, lr=LEARNING_RATE, weight_decay=0.01)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=NUM_EPOCHS, eta_min=1e-6
)

In [37]:
class EarlyStopping:
    """주어진 patience 후 검증 점수가 향상되지 않으면 학습을 조기 중단시킵니다."""

    def __init__(
        self,
        patience=7,
        verbose=False,
        delta=0,
        mode="min",
        path="../experiments/best_model.pt",
        evaluation_name="score",
    ):
        """
        Args:
            patience (int): 검증 점수가 향상된 후 기다릴 에폭 수.
            verbose (bool): True일 경우, 점수가 향상될 때마다 메시지를 출력.
            delta (float): 점수 향상으로 인정될 최소 변화량.
            mode (str): 'min' 또는 'max'. min은 점수가 낮아지는 것을, max는 높아지는 것을 목표로 함.
            path (str): 모델 체크포인트 저장 경로.
            evaluation_name (str): 로그에 표시될 평가 지표의 이름.
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.path = path
        self.mode = mode
        self.evaluation_name = evaluation_name

        if self.mode == "min":
            self.val_score = np.inf
        else:
            self.val_score = -np.inf

    def __call__(self, score, model):

        # mode에 따른 점수 비교
        is_best = False
        if self.mode == "min":
            if (
                score < self.best_score - self.delta
                if self.best_score is not None
                else True
            ):
                is_best = True
        else:  # mode == 'max'
            if (
                score > self.best_score + self.delta
                if self.best_score is not None
                else True
            ):
                is_best = True

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(score, model)
        elif is_best:
            self.best_score = score
            self.save_checkpoint(score, model)
            self.counter = 0
        else:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

    def save_checkpoint(self, score, model):
        """Saves model when validation score improves."""
        if self.verbose:
            # mode에 따라 'decreased' 또는 'increased'를 동적으로 표현
            change_text = "decreased" if self.mode == "min" else "increased"
            print(
                f"{self.evaluation_name} {change_text} ({self.val_score:.6f} --> {score:.6f}). Saving model ..."
            )

        save_dir = os.path.dirname(self.path)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            print(f"Created directory: {save_dir}")

        torch.save(model.state_dict(), self.path)
        self.val_score = score

In [38]:
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

backbone.body.conv1.weight: requires_grad=False
backbone.body.layer1.0.conv1.weight: requires_grad=False
backbone.body.layer1.0.conv2.weight: requires_grad=False
backbone.body.layer1.0.conv3.weight: requires_grad=False
backbone.body.layer1.0.downsample.0.weight: requires_grad=False
backbone.body.layer1.1.conv1.weight: requires_grad=False
backbone.body.layer1.1.conv2.weight: requires_grad=False
backbone.body.layer1.1.conv3.weight: requires_grad=False
backbone.body.layer1.2.conv1.weight: requires_grad=False
backbone.body.layer1.2.conv2.weight: requires_grad=False
backbone.body.layer1.2.conv3.weight: requires_grad=False
backbone.body.layer2.0.conv1.weight: requires_grad=True
backbone.body.layer2.0.conv2.weight: requires_grad=True
backbone.body.layer2.0.conv3.weight: requires_grad=True
backbone.body.layer2.0.downsample.0.weight: requires_grad=True
backbone.body.layer2.1.conv1.weight: requires_grad=True
backbone.body.layer2.1.conv2.weight: requires_grad=True
backbone.body.layer2.1.conv3.wei

In [39]:
from torchvision.ops import box_iou
# 3. 학습 루프
print("--- Start Training ---")
# ⭐️⭐️⭐️⭐️⭐️
metric = MeanAveragePrecision(box_format="xyxy", class_metrics=True).to(DEVICE)
early_stopping = EarlyStopping(
    patience=10, verbose=True, mode="max", evaluation_name="Validation mAP", path="../experiments/"+MODEL_NAME
)
# ⭐️⭐️⭐️⭐️⭐️

train_losses = []
val_losses = []
for epoch in range(NUM_EPOCHS):
    # ===================================
    #  Training Step
    # ===================================

    model.train()
    running_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{NUM_EPOCHS}]")

    for images, targets in loop:
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        running_loss += losses.item()
        current_avg_loss = running_loss / (loop.n + 1)
        loop.set_postfix(loss=current_avg_loss)

    avg_train_loss = running_loss / len(train_loader)

    # =====================================
    #  Validation Step (✨ 여기가 핵심 수정)
    # =====================================
    model.eval()
    val_loss = 0.0
    total_iou = 0
    tp_count = 0

    metric.reset()
    # (2) Validation phase
    for images, targets in val_loader:
        images = [img.to(DEVICE) for img in images]
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        # 1. mAP 계산을 위한 예측 (그래디언트 계산 불필요)
        with torch.no_grad():  # 👈✨ 예측 부분만 no_grad로 감싸기
            predictions = model(images)

        # 2. Metric 업데이트
        metric.update(predictions, targets)

        # ⭐️⭐️⭐️⭐️⭐️
        # IoU 평가 메트릭
        for i in range(len(predictions)):
            pred_boxes = predictions[i]["boxes"]
            gt_boxes = targets[i]["boxes"]

            if len(pred_boxes) == 0 or len(gt_boxes) == 0:
                continue

            # 각 GT 박스에 대해 가장 IoU가 높은 예측 박스를 찾음
            iou_matrix = box_iou(pred_boxes, gt_boxes)
            max_iou_per_gt, _ = iou_matrix.max(dim=0)

            # IoU가 0.5 이상인 경우 (TP)만 계산에 포함
            true_positives = max_iou_per_gt[max_iou_per_gt > 0.5]

            if len(true_positives) > 0:
                total_iou += true_positives.sum().item()
                tp_count += len(true_positives)
        # ⭐️⭐️⭐️⭐️⭐️

        # 3. Validation Loss 계산 (그래디언트 계산 필요)
        #    torch.no_grad() 블록 바깥에서 계산
        model.train()  # Loss 계산을 위해 잠시 train 모드로
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        val_loss += losses.item()
        model.eval()  # 다음 배치를 위해 다시 eval 모드로 복귀

    average_val_loss = val_loss / len(val_loader)
    val_losses.append(average_val_loss)

    # mAP 평가
    mAP_dict = metric.compute()
    # Learning Rate Scheduler 적용. val_loss를 고려하여 learning rate를 조정

    # mAP 기반EarlyStopping 로직 호출
    early_stopping(mAP_dict["map"], model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    # scheduler 설정
    scheduler.step()
    current_lr = optimizer.param_groups[0]["lr"]

    loop.write(
        f"Train Loss: {avg_train_loss:.4f}, Val Loss: {average_val_loss:.4f}, LR: {current_lr:.0e}"
    )
    loop.write(
        f"COCO mAP: {mAP_dict['map']:.4f}, mAP@50: {mAP_dict['map_50']:.4f}, mAP@75: {mAP_dict['map_75']:.4f}"
    )
    # loop.write(
    #     f"Val_Cls: {val_losses['loss_classifier']:.4f}, Val_Box: {val_losses['loss_box_reg']:.4f} | "
    # )
    # 메모리 정리
    gc.collect()  # 남아 있는 텐서 객체를 명시적으로 정리
    # torch.cuda.empty_cache()

print("--- Finish Training ---"),
# 최종 모델 저장
# torch.save(model.state_dict(), f"{EXPERIMENT_DIR}/final_model.pt")

--- Start Training ---


Epoch [1/20]: 100%|██████████| 58/58 [01:12<00:00,  1.25s/it, loss=0.758]


Validation mAP increased (-inf --> 0.012876). Saving model ...
Created directory: ../experiments
Train Loss: 0.7582, Val Loss: 0.8660, LR: 1e-04
COCO mAP: 0.0129, mAP@50: 0.0239, mAP@75: 0.0113


Epoch [2/20]: 100%|██████████| 58/58 [01:12<00:00,  1.25s/it, loss=0.848]


Validation mAP increased (0.012876 --> 0.122215). Saving model ...
Train Loss: 0.8477, Val Loss: 0.6746, LR: 1e-04
COCO mAP: 0.1222, mAP@50: 0.1864, mAP@75: 0.1480


...

Epoch [19/20]: 100%|██████████| 58/58 [01:18<00:00,  1.35s/it, loss=0.0853]


Validation mAP increased (0.575360 --> 0.579204). Saving model ...
Train Loss: 0.0853, Val Loss: 0.1143, LR: 2e-06
COCO mAP: 0.5792, mAP@50: 0.6201, mAP@75: 0.6201


Epoch [20/20]: 100%|██████████| 58/58 [01:17<00:00,  1.34s/it, loss=0.0838]


Validation mAP increased (0.579204 --> 0.579705). Saving model ...
Train Loss: 0.0838, Val Loss: 0.1108, LR: 1e-06
COCO mAP: 0.5797, mAP@50: 0.6191, mAP@75: 0.6191
--- Finish Training ---


(None,)